<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex10/patrick_ferreira/ex10_patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de referência 

Nome: Patrick de Carvalho Tavares Rezende Ferreira

## Instruções:

O objetivo desse colab é entender o comportamento das seguintes variáveis importantes de treinamento:
   - Batch size
   - Learning rate
   - FLOPs (computação gasta no treinamento)
   - Tamanho do modelo

Para tanto, iremos treinar e medir a loss e acurácia de 3 modelos BERT para análise de sentimento (classificação binária) usando o dataset do IMDB (20k/5k amostras de treino/validação).

Iremos fazer os 3 x 5 x 5 = 75 treinamentos, cada um usando um modelo (dentre 3), uma learning rate (dentre 5 valores) e batch size (dentro 5 valores) diferentes.

Os modelos sugeridos a serem usados são:

*   google/bert_uncased_L-2_H-128_A-2 (BERT-tiny, 4M params, ~0.5M non-embeddings)
*   google/bert_uncased_L-4_H-256_A-4 (BERT-mini, 11M params, ~3.5M non-embeddings)
*   google/bert_uncased_L-8_H-512_A-8 (BERT-medium, 41M params, ~25M non-embeddings)

Durante cada treinamento, iremos armezenar as seguntes informações: 

    - GPU usada
    - FP16 ou 32?
    - step atual
    - tempo de treinamento até então (wall time)
    - loss de treino
    - loss de validação
    - acurácia de validação
    
Iremos gravar essas informações _várias vezes por época_. Caso os treinamentos usem GPUs diferentes, podemos ajustar o wall time com base no FLOPs das GPUs.

Ao final, iremos plotar os seguintes gráficos:

1.   batch_size vs learning rate vs melhor loss de validação para cada modelo (usar gráfico 3D ou heatmap);
2.   tempo de treinamento (wall time) vs loss de validação. Plotar uma série (curva) para cada modelo, todas no mesmo gráfico. Para gerar cada curva, usar os melhores batch size e learning rate encontrados no gráfico 1. 

Com isso conseguiremos responder às seguintes perguntas:

    - Se você tiver T horas de GPU para usar, é melhor usar o modelo tiny, mini ou medium? Verifique se existe alguma faixa de valores de T em que é melhor usar o tiny. 
    - Qual modelo demora mais para atingir a sua melhor acurácia de validação em termos de épocas. E em termos de tempo de treino, wall time?
    - Para cada X vezes que aumentamos o batch size, como que devemos ajustar a learning rate?
    - Os melhores hiperparametros são parecidos para os 3 modelos?

Notas:
- Para entender melhor como batch size e learning rate se relacionam, procure fazer a varredura com passos de 5x ou 10x. Por exemplo:
    
    learning rate = {1e-2, 1e-3, ..., 1e-6}
    
    batch size = {1, 10, 100, 1000, 10000}

- Caso o batch não caiba em memória, usar acumulo de gradiente.
- Tempos estimados de treinamento para uma época do IMDB usando uma T4:
    - BERT-tiny: menos de 1 minuto
    - BERT-mini: 3 minutos
    - BERT-medium: 10 minutos. Portanto, se treinarmos por 2 épocas, o tempo total para rodar os experimentos será de `2 épocas x 10 min x 25 treinamentos ~ 9 horas`.
- Sugerimos fazer primeiro todos os experimentos com BERT-tiny e BERT-mini. Quando souber da faixa de hiperparametros "bons", não precisa fazer os 25 treinamentos para o BERT-medium.
    - TFLOPs (FP32) de cada GPU:
        T4: 8,141
        K80: 4,113
        A100: 19,49
    - Usar time.perf_counter() para medir o wall time.

# Fixando a seed

In [1]:
import random
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from typing import List, Type
from tqdm.notebook import tqdm

In [2]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

In [4]:
# Check which GPU we are using
!nvidia-smi

Tue Nov  1 14:57:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P0    N/A /  N/A |    152MiB /  2004MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Preparando Dados

Primeiro, fazemos download do dataset:

In [6]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (20k exemplos) e validação (5k exemplos) artificialmente.

In [7]:
import os

max_valid = 5000

def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [1] * len(x_train_pos) + [0] * len(x_train_neg)
y_test = [1] * len(x_test_pos) + [0] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

x_valid = x_train[-max_valid:]
y_valid = y_train[-max_valid:]
x_train = x_train[:-max_valid]
y_train = y_train[:-max_valid]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('3 primeiras amostras validação:')
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print('3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])


20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
0 A truly dreadful film. I did not know initially that this was a Kiwi effort - but very soon I starte
0 well its official. they have just killed American Pie. The first 3 were absolutely hysterical, but t
1 I've read some terrible things about this film, so I was prepared for the worst. "Confusing. Muddled
3 últimas amostras treino:
0 I cannot stand this show! Has there ever been even one redeeming quality, one funny punchline, or on
1 Exquisite comedy starring Marian Davies (with the affable William Haines). Young Peggy arrives in Ho
1 Made me wish my own happy birds could talk. Tisk tisk on the reviewer who dissed the movie. A sweet 
3 primeiras amostras validação:
1 Oh my, from the box description I thought it would be LA-crazy like 2 Days in the Valley or Hugo Poo
1 Sure this movie wasn't like. 16 blocks, inside man, an American haunting. etc...<br /><br />But It w
1 B

In [8]:
### Criando classe do dataset

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, texts: List[str], labels):
        super().__init__()

        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], torch.tensor([0., 1.]) if self.labels[idx] else torch.tensor([1., 0.])

Dados de treino, validação e teste

In [9]:
training_dataset = MyDataset(x_train, y_train)
valid_dataset = MyDataset(x_valid, y_valid)
test_dataset = MyDataset(x_test, y_test)

print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 20000
valid examples: 5000
test examples: 25000


Testando se o modelo processa os dados corretamente

In [10]:
model = tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.train().to(device)

sample_train, _ = next(iter(DataLoader(training_dataset, batch_size=4)))

sample_train = tokenizer.batch_encode_plus(sample_train, padding=True, return_tensors="pt", truncation=True, max_length=200).to(device)

print("model output shape: ", model(**sample_train).logits.shape)

del sample_train


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

model output shape:  torch.Size([4, 2])


TREINAMENTO

In [11]:


from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast

max_examples = 20_000
eval_every_steps = 1000
lr = 4e-5
use_amp = True

train_loader = DataLoader(training_dataset, batch_size=2, shuffle=True, num_workers=1)
validation_loader = DataLoader(valid_dataset, batch_size=2, num_workers=1, )

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.9, min_lr=3e-5, patience=15000, threshold=1e-1, verbose=True)
scaler=GradScaler()


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    with autocast(enabled=use_amp):
        logits = model(**input_ids).logits
        logits = logits.reshape(-1, logits.shape[-1])
    loss = nn.functional.cross_entropy(logits, target_ids, )
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    accuracy = 0
    with autocast(enabled=use_amp):
        logits = model(**input_ids).logits
        logits = logits.reshape(-1, logits.shape[-1])
        loss = nn.functional.cross_entropy(logits, target_ids,)
        preds = logits.argmax(dim=1)
        accuracy += (preds == y).sum() / logits.shape[0]
    return loss.item(), accuracy

best_validation_loss = 9999
train_losses = []
n_examples = 0
step = 0
pbar = tqdm(total=max_examples)
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        train_input_ids = tokenizer.batch_encode_plus(train_input_ids, padding=True, return_tensors="pt", truncation=True, max_length=200).to(device)
        loss = train_step(train_input_ids, train_target_ids.to(device))
        train_losses.append(loss)

        # LR scheduler
        scheduler.step(loss)

        if step % eval_every_steps == 0:
            train_loss = np.average(train_losses)

            with torch.no_grad():
                valid_loss = np.average([
                    validation_step(tokenizer.batch_encode_plus(val_input_ids, padding=True, return_tensors="pt", truncation=True, max_length=200).to(device), val_target_ids.to(device))[0]
                    for val_input_ids, val_target_ids in validation_loader])
                # Checkpoint to best models found.
                if best_validation_loss > valid_loss:
                    # Update the new best perplexity.
                    best_validation_loss = valid_loss
                    model.eval()
                    torch.save(model, "best_model.pth")

            print(f'{step} steps; {n_examples} examples so far; train loss: {train_loss:.2f}, valid loss: {valid_loss:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        pbar.update(len(train_input_ids))
        if n_examples >= max_examples:
            break

pbar.close()

# Restore best model (checkpoint) found
model = torch.load("best_model.pth")


RuntimeError: multi-target not supported at /opt/conda/conda-bld/pytorch_1616554827596/work/aten/src/THCUNN/generic/ClassNLLCriterion.cu:15

Avaliação no dataset de Teste

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1024)

with torch.no_grad():
    losses = [
        validation_step(tokenizer.batch_encode_plus(input, padding=True, return_tensors="pt", truncation=True, max_length=200).to(device), target.to(device))
        for input, target in tqdm(test_loader)
    ]




In [ ]:
losses = list(zip(*losses))

test_loss, test_accuracy = np.average(losses[0]), torch.mean(torch.tensor(losses[1]).float())

print(f'test loss: {test_loss}', f'test acc: {test_accuracy}')